
# Illinois Professional Licenses ? Altair Workflow
This notebook loads the Illinois Department of Financial and Professional Regulation license records directly from GitHub, cleans a few high-value fields, and exports Vega-Lite JSON specs so the Jekyll site can embed the plots without re-running Python.



## Dataset & Plan
- **Dataset**: [licenses_fall2022.csv](https://github.com/UIUC-iSchool-DataViz/is445_data/raw/main/licenses_fall2022.csv)
- **Goal**: build two complementary views of the same data?one static summary of active licenses by type and one interactive time trend for the busiest professions.
- **Workflow**: load from the URL, standardize categorical text, parse the original issue date, engineer an `issue_year` helper column, and then export each Altair chart to `assets/json/` for the website.


In [ ]:

import pandas as pd
import altair as alt
from pathlib import Path

alt.data_transformers.disable_max_rows()

DATA_URL = "https://github.com/UIUC-iSchool-DataViz/is445_data/raw/main/licenses_fall2022.csv"

export_dir = Path.cwd() / "assets" / "json"
if not export_dir.exists():
    export_dir = Path.cwd().parent / "assets" / "json"
export_dir.mkdir(parents=True, exist_ok=True)

licenses = pd.read_csv(DATA_URL)
licenses.columns = [col.strip() for col in licenses.columns]
licenses['License Status'] = licenses['License Status'].fillna('Unknown').str.upper()
licenses['License Type'] = licenses['License Type'].fillna('Unspecified')
licenses['City'] = licenses['City'].fillna('Unknown')

for date_col in ['Original Issue Date', 'Effective Date', 'Expiration Date']:
    licenses[date_col] = pd.to_datetime(licenses[date_col], errors='coerce')

licenses['issue_year'] = licenses['Original Issue Date'].dt.year
licenses_clean = licenses.copy()
licenses_clean[['License Type', 'License Status', 'issue_year']].head()



## Visualization 1 ? Active Licenses by Profession
Count active credentials per license type and focus on the top ten categories.


In [ ]:

active = licenses_clean[licenses_clean['License Status'] == 'ACTIVE']
top_types = (
    active.groupby('License Type')
    .size()
    .reset_index(name='count')
    .sort_values('count', ascending=False)
    .head(10)
)

bar_chart = (
    alt.Chart(top_types)
    .mark_bar(color='#50a9d1')
    .encode(
        x=alt.X('count:Q', title='Number of Active Licenses'),
        y=alt.Y('License Type:N', sort='-x', title='License Type'),
        tooltip=['License Type:N', alt.Tooltip('count:Q', title='Active Licenses')]
    )
    .properties(width=600, title='Most Common Active License Categories (Top 10)')
)

bar_chart.save(str(export_dir / 'licenses_active_by_type.json'))
bar_chart



## Visualization 2 ? Interactive Issue Trends for Top Professions
Show how licensing volume changes through time for the busiest professions and let viewers pick the profession from a dropdown (beyond the default pan/zoom interactions).


In [ ]:

top5 = (
    licenses_clean.groupby('License Type')
    .size()
    .reset_index(name='count')
    .sort_values('count', ascending=False)
    .head(5)
    ['License Type']
)

issue_trends = (
    licenses_clean[
        licenses_clean['License Type'].isin(top5)
        & licenses_clean['issue_year'].notna()
    ]
    .groupby(['issue_year', 'License Type'])
    .size()
    .reset_index(name='count')
)

lic_options = top5.tolist()
selector = alt.selection_point(
    fields=['License Type'],
    bind=alt.binding_select(options=lic_options, name='License Type')
)

trend_chart = (
    alt.Chart(issue_trends)
    .mark_line(point=True)
    .encode(
        x=alt.X('issue_year:O', title='Original Issue Year'),
        y=alt.Y('count:Q', title='Licenses Issued'),
        color=alt.Color('License Type:N', legend=None),
        opacity=alt.condition(selector, alt.value(1.0), alt.value(0.15)),
        tooltip=['License Type:N', 'issue_year:O', alt.Tooltip('count:Q', title='Licenses Issued')]
    )
    .add_params(selector)
    .properties(width=600, title='Issue Volume for Top Illinois License Categories')
)

trend_chart.save(str(export_dir / 'licenses_issue_trends.json'))
trend_chart
